In [67]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import PGVector
import os
import warnings
warnings.filterwarnings('ignore')

In [39]:
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
import pandas as pd
cleaned_file_path = r"C:\Users\saigi\PycharmProjects\bookRecommender\data\books_cleaned.csv"
books = pd.read_csv(cleaned_file_path)

In [41]:
books[["isbn13","tagged_description"]].head()

,isbn13,tagged_description
0,9780002005883,0002005883 A NOVEL THAT READERS and critics ha...
1,9780002261982,0002261987 A new 'Christie for Christmas' -- a...
2,9780006178736,"0006178731 A memorable, mesmerizing heroine Je..."
3,9780006280897,0006280897 Lewis' work on the nature of love d...
4,9780006280934,"0006280935 ""In The Problem of Pain, C.S. Lewis..."


In [42]:
# books["tagged_description"].to_csv("tagged_description.csv", sep='\n', header=False ,index=False)

In [43]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0,chunk_overlap=0,separator='\n')
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1165, which is longer than the specified 0
Created a chunk of size 1211, which is longer than the specified 0
Created a chunk of size 370, which is longer than the specified 0
Created a chunk of size 306, which is longer than the specified 0
Created a chunk of size 480, which is longer than the specified 0
Created a chunk of size 479, which is longer than the specified 0
Created a chunk of size 957, which is longer than the specified 0
Created a chunk of size 185, which is longer than the specified 0
Created a chunk of size 840, which is longer than the specified 0
Created a chunk of size 293, which is longer than the specified 0
Created a chunk of size 194, which is longer than the specified 0
Created a chunk of size 878, which is longer than the specified 0
Created a chunk of size 1085, which is longer than the specified 0
Created a chunk of size 1186, which is longer than the specified 0
Created a chunk of size 301, which is longer than the specified 0
Create

In [44]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='0002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilea

In [48]:
CONNECTION_STRING = os.getenv('DATABASE_URL')

In [49]:
ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

In [50]:
COLLECTION_NAME = 'books_description_vectors'

db_books = PGVector.from_documents(
    embedding=ollama_embeddings,
    documents=documents,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

C:\Users\saigi\PycharmProjects\bookRecommender\.br_venv\Lib\site-packages\langchain_community\vectorstores\pgvector.py:487: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(


In [68]:
retriever = db_books.as_retriever(search_kwargs={'k': 3})  # default 4

retriever.get_relevant_documents("A book to teach children about nature")

[Document(metadata={'source': 'tagged_description.txt'}, page_content='"006757520X First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson views as essential to life. In her insightful n

In [66]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='"006757520X First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson views as essential to life. In her insightful n

In [62]:
retrived_str=docs[0].page_content.split()[0].strip().replace('"', '')
# retrived_str = retrived_str.replace('"', '')
books[books["isbn10"] == retrived_str]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,006757520X First published more than three dec...


In [70]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [(recs[i].page_content.strip('"').split()[0].replace('"', '').replace("'", ''))]

    return books[books["isbn10"].isin(books_list)]

In [71]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
31,9780007105045,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"0007105045 ""The two works 'On fairy-stories' a..."
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,0007151241 It should be the most natural thing...
86,9780060000141,0060000147,Poppy's Return,Avi,Juvenile Fiction,http://books.google.com/books/content?id=XbcMJ...,"There's trouble at Gray House, the girlhood ho...",2006.0,3.99,256.0,1086.0,Poppy's Return,"0060000147 There's trouble at Gray House, the ..."
107,9780060277406,0060277408,The Secret Garden Cookbook,Amy Cotler,Juvenile Nonfiction,http://books.google.com/books/content?id=c7E_H...,Frances Hodgson Burnett's The Secret Garden de...,1999.0,4.28,128.0,142.0,The Secret Garden Cookbook: Recipes Inspired b...,0060277408 Frances Hodgson Burnett's The Secre...
146,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,"The Lion, the Witch and the Wardrobe (picture ...",0060556501 Narnia: A magical land full of wond...
223,9780060775858,0060775858,Goodnight Moon 60th Anniversary Edition,Margaret Wise Brown,Juvenile Fiction,http://books.google.com/books/content?id=lLYOr...,"In a great green room, tucked away in bed, is ...",2005.0,4.27,32.0,264013.0,Goodnight Moon 60th Anniversary Edition,"0060775858 In a great green room, tucked away ..."
267,9780060882600,0060882603,The Annotated Charlotte's Web,E. B. White,Juvenile Nonfiction,http://books.google.com/books/content?id=vaYYH...,"Charlotte's Web, one of America's best-loved c...",2006.0,4.16,320.0,41.0,The Annotated Charlotte's Web,"0060882603 Charlotte's Web, one of America's b..."
274,9780060887964,0060887966,The Alchemist - Gift Edition,Paulo Coelho,Fiction,http://books.google.com/books/content?id=h6QuJ...,"""My heart is afraid that it will have to suffe...",2006.0,3.85,192.0,814.0,The Alchemist - Gift Edition,"0060887966 ""My heart is afraid that it will ha..."
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,0060959037 Barbara Kingsolver's fifth novel is...
366,9780061122415,0061122416,The Alchemist,Paulo Coelho,Fiction,http://books.google.com/books/content?id=pTr44...,"""My heart is afraid that it will have to suffe...",2006.0,3.85,197.0,1592632.0,The Alchemist,"0061122416 ""My heart is afraid that it will ha..."
